In [3]:
import numpy as np
import os
# matplotlib 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### 데이터 다운받기

In [4]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [5]:
X_train.shape
# y_train

(60000, 28, 28)

In [17]:
X_train.shape # 60000, 28, 28(eigen vector로 이미 바꿈)
X_train = X_train.reshape(60000, 28*28)
X_train.shape
# y_train.shape #1797

some_digit = X_train[36000]
some_digit
# plt.imshow(some_digit)
y_train[36000]

9

In [18]:
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

## 이진분류기 훈련

In [19]:
y_train_9 = (y_train == 9) # 5는 true이고 나머지는 false
y_test_9 = (y_test == 9)
y_train_9

array([False, False, False, ..., False, False, False])

In [20]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, random_state=42)
sgd_clf.fit(X_train, y_train_9)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_9, cv=3, scoring="accuracy")

array([0.95185, 0.9432 , 0.92805])

In [26]:
from sklearn.base import BaseEstimator

class Never9Classfier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len (X), 1), dtype=bool)

In [28]:
never_9_clf = Never9Classfier()
cross_val_score(never_9_clf, X_train, y_train_9, cv=3, scoring="accuracy")

array([0.90195, 0.9025 , 0.8981 ])

## 오차행렬 (CONFUSION MATRIX)

In [31]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_9, cv=3) # 점수 반환이 아니라 예측 반환

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_9, y_train_pred)

array([[53071,   980],
       [ 2558,  3391]], dtype=int64)

## 정밀도(precision)과 재현율(recall)

In [36]:
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_train_9, y_train_pred)) # 9로 판별된 이미지 중에서 77퍼만 정확
print(recall_score(y_train_9, y_train_pred)) # 전체 숫자 9에서 57퍼만 감지

0.7757950125829329
0.5700117666834762


In [37]:
# 정밀도와 재현율을 조화평균으로 f1_score만듬
from sklearn.metrics import f1_score
f1_score(y_train_9, y_train_pred)

0.6571705426356588

In [38]:
# 각 샘플의 점수를 얻어 임곗값을 정해 예측을 만든다.